In [1]:
# set dependencies
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import json
import time
import gmaps

from pprint import pprint
from scipy import stats
from scipy.stats import linregress
from citipy import citipy

# Set options
pd.options.display.max_rows=601
pd.set_option("precision", 2)
#from config import api_key

In [2]:
# scatter plot function
def sbc_plot_scatter(df,col1,col2,title,file):
    x=df[col1].astype('float') # Key error is argument conflict
    y=df[col2].astype('float') # Key error is argument conflict     
    (slope,intercept,rvalue,pvalue,stderr)=stats.linregress(x,y)
    fit=x*slope+intercept
    line='y = '+str(round(slope,2))+'x + '+str(round(intercept,2))
    x_min=round(df[col1].min()/10)*10 # key error
    x_max=(round(df[col1].max()/10)*10)+10 # key error
    plt.xticks(np.arange(x_min,x_max,10))
    plt.xlabel(col1)
    plt.ylabel(col2)
    plt.grid(b=True)
    plt.scatter(x,y)
    plt.title(title)
    plt.plot(x,fit,color='red')
    plt.annotate(line,(47,10),color='red',xycoords='figure pixels')
    plt.annotate((f"R squared: {round((rvalue**2),2)}"),(299,10),color='red',xycoords='figure pixels')
    plt.savefig(f'dataFiles/output/{file}.png') # must conform to your local repo file path or get file or directory not found
                                                # error
    
# how to call the function:
# sbc_plot_scatter(df_name,'<col1_header>','<col2_header>','<your title text>','<your_output_file_name')
#                  no quotes,rest use quotes-----------------------------------------------------------|
# Key errors indicate the function call could not find the column in the dataframe
# syntax errors indicate missing commas or quotes
# attribute errors indicate the df_name item is not a DataFrame object

In [6]:
# Create dataframe from csv - This csv is the list of winners of post season NFL games 1970-2021.
SK_bday_loc = "dataFiles/PossibleData/CSV/SK_bday_loc_data_CSV.csv"
SK_bday_loc = pd.read_csv(SK_bday_loc)
SK_bday_loc.head()

,Serial Killer Name,Gender,Birthday,Birth Year,Killing Avg Start,Stat Year of First Killing,Country Active,Years Active,Proven Victims,Possible Victims,Unsolved Murders,Type,Type Specific,Nickname,Notes
0,Harold Shipman,M,1/14/1946,1946.0,29.0,1975.0,United Kingdom,1975-1998,218.0,250.0,32.0,Medical,NaN,Dr Death Angel of Death,Convicted of 15 murders and responsible for th...
1,Miyuki Ishikawa,F,1897,1897.0,31.0,1928.0,Japan,1940,103.0,169.0,66.0,Medical,NaN,NaN,Killed more than 103 newborn children. As a ma...
2,Niels Högel,M,12/30/1976,1976.0,29.0,2005.0,Germany,1999-2005,100.0,300.0,200.0,Medical,NaN,NaN,Nurse who was sentenced to life imprisonment f...
3,Louay Omar Mohammed al-Taei,M,NaN,NaN,NaN,0.0,Iraq,2005-2006,43.0,0.0,NaN,Medical,NaN,NaN,Medical doctor found to have killed 43 wounded...
4,Donald Harvey,M,4/15/1952,1952.0,29.0,1981.0,United States,1970-1987,37.0,77.0,40.0,Medical,NaN,Angel of Death,Self-professed Angel of Death. Worked as an or...


In [5]:
# Create dataframe from csv .
SK_most_victims = "dataFiles/PossibleData/CSV/SK_most_victims_CSV.csv"
SK_most_victims = pd.read_csv(SK_most_victims)
SK_most_victims.head()

,Serial Killer Name,Gender,Birthday,Birth Year,Killing Avg Start,Stat Year of First Killing,Country Active,Years Active,Years Killing,Proven victims,IQ Level,Notes
0,Luis Garavito,M,1/25/1957,1957.0,29.0,1986.0,Colombia,1992-1999,7,138,NaN,"Child-murderer, torture-killer, and rapist kno..."
1,Pedro López,M,10/8/1948,1948.0,29.0,1977.0,Colombia,1969-1979,10,110,NaN,"Child-murderer and rapist, known as ""The Monst..."
2,Javed Iqbal,M,10/8/1956,1956.0,29.0,1985.0,Pakistan,1998-1999,1,100,NaN,"Child-murderer and rapist, known as ""Kukri"", I..."
3,Mikhail Popkov,M,3/7/1964,1964.0,29.0,1993.0,Russia,1992-2010,18,78,NaN,"Serial rapist-killer nicknamed ""The Werewolf"",..."
4,Daniel Camargo Barbosa,M,1/22/1930,1930.0,29.0,1959.0,Colombia,1974-1986,12,72,NaN,"Child and woman murderer, believed to have pos..."


In [7]:
# Create dataframe from csv .
US_crime = "dataFiles/PossibleData/CSV/1997-2016_US_crime_CSV.csv"
US_crime = pd.read_csv(US_crime)
US_crime.head()

,Table 1,,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57
0,Crime in the United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"by Volume and Rate per 100,000 Inhabitants, 19...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Year,Population1,Violent\ncrime2,Violent \ncrime \nrate,Murder and\nnonnegligent \nmanslaughter,Murder and \nnonnegligent \nmanslaughter \nrate,Rape\n(revised \ndefinition3),Rape\n(revised \ndefinition) \nrate3,Rape\n(legacy \ndefinition4),Rape\n(legacy \ndefinition) \nrate4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1997,"267,783,607","1,636,096",611.0,"18,208",6.8,NaN,NaN,"96,153",35.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1998,"270,248,003","1,533,887",567.6,"16,974",6.3,NaN,NaN,"93,144",34.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Create dataframe from csv .
SK_active = "dataFiles/PossibleData/CSV/active_SK_year_CSV.csv"
SK_active = pd.read_csv(SK_active)
SK_active.head()

,Year,Female,All,US
0,1900,5,8,2
1,1903,3,3,2
2,1906,3,6,2
3,1909,2,7,7
4,1912,4,10,3


In [11]:
Ratsford = "dataFiles/PossibleData/CSV/Ratsford_university_sk_CSV.csv"
Ratsford =  pd.read_csv(Ratsford)
Ratsford.head()

,Country,Number of Serial Killers,% of Serial Killers,Country's Share of World Population,Ratio
0,US,"3,204",67.58,4.35%,15.53
1,England,166,3.50,0.71%,4.92
2,S. Africa,117,2.47,0.74%,3.34
3,Canada,106,2.24,0.49%,4.59
4,Italy,97,2.05,0.80%,2.55


In [13]:
SK_state = "dataFiles/PossibleData/CSV/SK_by_state_2021.csv"
SK_state =  pd.read_csv(SK_state)
SK_state.head()

,State,serialKillersVictims,serialKillersVictimsPerCapita,Pop
0,Alaska,51,7.04,724357
1,Louisiana,300,6.48,4627000
2,Kansas,153,5.24,2917220
3,Missouri,311,5.04,6169040
4,Indiana,341,5.01,6805660


In [15]:
killer_age = "dataFiles/PossibleData/CSV/killer_by_age_CSV.csv"
killer_age =  pd.read_csv(killer_age)
killer_age.head()

,Unnamed: 0,Male,Female,N/A
0,Total (%),546.0,75.0,30.0
1,Death Date Age,49.0,58.0,NaN
2,First Kill Age,29.0,31.0,NaN
3,Last Kill Age,35.0,38.0,NaN
4,Arrested,37.0,39.0,NaN


In [16]:
killer_month = "dataFiles/PossibleData/CSV/killer_by_month_CSV.csv"
killer_month =  pd.read_csv(killer_month)
killer_month.head()

,Month,Killers,Unnamed: 2
0,Jan,56.0,NaN
1,Feb,38.0,NaN
2,March,38.0,NaN
3,Apr,38.0,NaN
4,May,0.0,NaN


In [17]:
killer_why = "dataFiles/PossibleData/CSV/killer_genders_why_kill_CSV.csv"
killer_why =  pd.read_csv(killer_why)
killer_why.head()

,Unnamed: 0,Male,Female,N/A,Both Genders
0,Poison (%),5.0,21.0,NaN,NaN
1,Strangle(%),NaN,21.0,NaN,NaN
2,Shoot (%),NaN,7.0,NaN,NaN
3,Kill for Sex (%),46.0,NaN,NaN,NaN
4,Kill for Control (%),29.0,13.0,NaN,NaN


In [18]:
killer_profile = "dataFiles/PossibleData/CSV/profile_of_killer_CSV.csv"
killer_profile =  pd.read_csv(killer_profile)
killer_profile.head()

,Serial Killer Demographics,True,FALSE,Demographic (%)
0,Previous Crimes,298.0,69.0,81.0
1,Rape,308.0,105.0,75.0
2,Used Weapon,318.0,135.0,70.0
3,Previous Jail,241.0,111.0,68.0
4,Strangled,251.0,192.0,57.0


In [19]:
killer_iq = "dataFiles/PossibleData/CSV/serial_killer_iq_CSV.csv"
killer_iq =  pd.read_csv(killer_iq)
killer_iq.head()

,Serial Killer Name,IQ Level,Gender,Unnamed: 3,Unnamed: 4
0,Charlene Gallego,160,F,https://killer.cloud/serial-killers/by/iq-leve...,NaN
1,Rodney Alcala,160,M,72 total,NaN
2,Carroll Edward Cole,152,M,NaN,NaN
3,Edmund Kemper,145,M,NaN,NaN
4,Robert Browne,140,M,NaN,NaN


# Merge Datasets

In [28]:
SK_merged=pd.merge(SK_bday_loc,SK_most_victims, on="Serial Killer Name", how= "left")
SK_merged.head()
SK_merged.to_csv("dataFiles/output/Working- SK Merged.csv")

In [ ]:
SK_all= SK_merged.rename(columns={"Gender_x":"Gender", })

In [ ]:
#sbc_plot_scatter(winners_df,'Winning Score','Super Bowl Appearances','Test','test_file') - Shows single and double quote.
#                                                                                           mixing quotes gives syntax error
sbc_plot_scatter(winners_df,"Winning Score","Super Bowl Appearances","Test","test_file")

In [ ]:
#gather data - Use API or import files and convert to dataframe - .txt, .csv, Excel, json, .py, I can line out steps to process
#                                                                 into dataframe
#ADD your url
#ADD your filepath (remember to push your file to the branch)

## CLEAN DATA

## PROCESS DATA

## ANALYZE RESULTS

## DETERMINE TRENDS

## COMPILE CONCLUSIONS

## FORMULATE RECOMMENDATIONS

### Summary